# Notebook 3: Autoaprendizaje

## Pre-requisitos

### Instalar paquetes

Si la práctica requiere algún paquete de Python, habrá que incluir una celda en la que se instalen. Si usamos un paquete que se ha utilizado en prácticas anteriores, podríamos dar por supuesto que está instalado pero no cuesta nada satisfacer todas las dependencias en la propia práctica para reducir las dependencias entre ellas.

In [3]:
# Ejemplo de instalación de tensorflow 2.0
#%tensorflow_version 2.x
# !pip3 install tensorflow  # NECESARIO SOLO SI SE EJECUTA EN LOCAL
import tensorflow as tf

# Hacemos los imports que sean necesarios
import numpy as np

# Autoaprendizaje sobre Fashion-MNIST

Lo primero que tenemos que hacer es cargar el dataset.

In [4]:
labeled_data = 0.05 # Vamos a usar el etiquetado de sólo el 5% de los datos
np.random.seed(42)

(x_train, y_train), (x_test, y_test), = tf.keras.datasets.fashion_mnist.load_data()

indexes = np.arange(len(x_train))
np.random.shuffle(indexes)
ntrain_data = int(labeled_data*len(x_train))
unlabeled_train = x_train[indexes[ntrain_data:]]
x_train = x_train[indexes[:ntrain_data]]
y_train = y_train[indexes[:ntrain_data]]

In [5]:
# TODO: Haz el preprocesado que necesites aquí (si lo necesitas)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1] * x_train.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))
unlabeled_train = np.reshape(unlabeled_train, (unlabeled_train.shape[0], unlabeled_train.shape[1] * unlabeled_train.shape[2]))
print(x_train.shape, x_test.shape, unlabeled_train.shape)

(3000, 784) (10000, 784) (57000, 784)


## Función de autoaprendizaje

Vamos a crear nuestra propia función de autoaprendizaje. Para ello, vamos a utilizar el siguiente pseudocódigo.



**self_training** *(model, x_train, y_train, unlabeled_data, thresh, train_epochs)*

1. $train\_data, train\_label \leftarrow x\_train, y\_train$
1. **Desde** $n = 1 .. train\_epochs$ **hacer**
	1. Entrena el *model* usando las variables *train\_data* y *train\_label* 
	2. $y\_pred \leftarrow model(unlabeled\_data)$
  
  3. $y\_class, y\_value \leftarrow $ Clase ganadora en *y_pred* con su valor

  4. $train\_data, train\_label \leftarrow x\_train, y\_train$
  
  5. **Para cada elemento** (x_u, y_c, y_v) **de la tupla** (unlabeled_data, y_class, y_value) 
	  1. **Si** $y\_v > thresh$ **entonces**
		
        1. Añadimos $x\_u$ e $y\_c$ a train\_data y train\_label, respectivamente.

4. Devolvemos el modelo


<font color='red'>NOTA:</font> para entrenar (y predecir) vamos a utilizar los modelos de Sklearn. Familiarízate con las funciones [fit](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.fit), [predict](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict), [predict_proba](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict_proba) y [score](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.score).

In [6]:
# TODO: implementa el algoritmo self_training tal y como viene en el pseudocódigo. Las variables extra son para epara visualización de resultados

def self_training(model_func, x_train, y_train, unlabeled_data, x_test, y_test, thresh=0.8, train_epochs=3):
    train_data = x_train.copy()
    train_label = y_train.copy()

    for i in range(train_epochs):
        model = model_func()
        model.fit(train_data, train_label)
        # Predecir en los datos no etiquetados
        # model.predict_proba() devuelve la probabilidad de cada clase para cada ejemplo 
        # (n_samples, n_classes)
        y_pred = model.predict_proba(unlabeled_data)
        # Necesitamos usar axis=1 para obtener el máximo de cada fila, es decir, de cada ejemplo
        # de esta forma obtenemos el valor de la clase predicha y la probabilidad de la clase predicha
        y_class, y_value = np.argmax(y_pred, axis=1), np.max(y_pred, axis=1)
        # Reseteamos los datos de entrenamiento
        train_data = x_train.copy()
        train_label = y_train.copy()
        
        # Añadimos los datos con probabilidad mayor que el thresh
        for i in range(len(unlabeled_data)):
            if y_value[i] > thresh:
                train_data = np.append(train_data, [unlabeled_data[i]], axis=0)
                train_label = np.append(train_label, [y_class[i]], axis=0)
        print(f"Epoch {i+1} - Added {len(train_data) - len(x_train)} samples")
    model = model_func()
    model.fit(train_data, train_label)
    return model    

### Entrenamos nuestro clasificador

Usa lo hecho anteriormente para entrenar tu clasificador de una manera semi-supervisada. Utiliza para ello el [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) de sklearn (vigila el parámetro probability).

In [ ]:
# Define la función para llamar al SVM
from sklearn.svm import SVC
model_func = lambda: SVC(kernel='linear', probability=True, max_iter=200)

In [6]:
# TODO: Entrena tu clasificador
model = self_training(model_func, x_train, y_train, unlabeled_train, x_test, y_test) 
print("Test Score: ", model.score(x_test, y_test))

Epoch 1


/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Epoch 2


/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Epoch 3


/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Test Score:  0.7973


## Mejorando el código

Tal como hemos visto en clase de teoría, este código puede ser mejorado de varias maneras:

  1. **Asignar más peso a las variables etiquetadas**.
  1. **Asignar un peso en función de la certeza de la predicción en las variables sin etiquetar**.

### TRABAJO: Modifica la función self_training para tener en cuenta todos los puntos mencionados anteriormente

In [ ]:
def self_training_v2(model_func, x_train, y_train, unlabeled_data, x_test, y_test, thresh=0.8, train_epochs=3):
    train_data = x_train.copy()
    train_label = y_train.copy()
    # Inicializamos los pesos de las muestras a 2.0
    sample_weights = np.ones(len(train_label)) * 2.0
    
    # Trabajamos directamente con unlabeled_data
    current_unlabeled = unlabeled_data.copy()
    
    for i in range(train_epochs):
        if len(current_unlabeled) == 0:
            print("No more unlabeled data left")
            break
            
        model = model_func()
        # Usamos los pesos de las muestras
        model.fit(train_data, train_label, sample_weight=sample_weights)
        
        # Predicción en datos sin etiquetar
        y_pred = model.predict_proba(current_unlabeled)
        y_class = np.argmax(y_pred, axis=1)  # Clase con mayor probabilidad
        y_value = np.max(y_pred, axis=1)     # Valor de probabilidad más alto
        
        # Seleccionar ejemplos con confianza superior al umbral
        high_confidence = y_value > thresh
        
        if np.any(high_confidence):
            # Obtener datos, etiquetas y probabilidades de los ejemplos de alta confianza
            new_data = current_unlabeled[high_confidence]
            new_labels = y_class[high_confidence]
            new_probs = y_value[high_confidence]
            
            # Añadir a datos de entrenamiento
            train_data = np.vstack([train_data, new_data])
            train_label = np.append(train_label, new_labels)
            sample_weights = np.append(sample_weights, new_probs)
            
            # Eliminar ejemplos usados
            current_unlabeled = current_unlabeled[high_confidence == False]
            
            print(f"Epoch {i+1}: {len(new_data)} samples added, {len(current_unlabeled)} remaining unlabeled")
        else:
            print(f"Epoch {i+1}: No samples added")
    
    # Entrenamos el modelo final
    model = model_func()
    model.fit(train_data, train_label, sample_weight=sample_weights)
    return model

In [14]:
# TODO: Entrena tu clasificador
model_func = lambda: SVC(kernel='linear', probability=True, max_iter=200)
model = self_training_v2(model_func, x_train, y_train, unlabeled_train, x_test, y_test) 
print("Test Score: ", model.score(x_test, y_test))

/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Epoch 1: 27477 samples added, 29523 remaining unlabeled


/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Epoch 2: 8083 samples added, 21440 remaining unlabeled


/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Epoch 3: 5634 samples added, 15806 remaining unlabeled


/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Test Score:  0.7852


### Creamos nuestro clasificador en Tensorflow

Ya deberías de ser capaz de realizar este trabajo con muy poca ayuda. El diseño del clasificador es libre (capas densas, convolucionales, ...), puedes crearlo como quieras. Lo único que tenemos que tener en cuenta es que tenemos que encapsular nuestro modelo en una clase cuyas funciones tengan el mismo nombre (y los mismos parámetros) que las existentes en sklearn.

In [36]:
# TODO: crea tu propio clasificador

class MiClasificador:

    def __init__(self):
        # TODO : define el modelo
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
        # TODO: crea el optimizador
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.loss = tf.keras.losses.SparseCategoricalCrossentropy()
        # TODO: compila el modelo
        self.model.compile(
            loss=self.loss,
            optimizer=self.optimizer,
            metrics=['accuracy']
        )
    
    def fit(self, X, y, sample_weight=None):
        # TODO: entrena el modelo. Escoge el tamaño de batch y el número de epochs que quieras
        self.model.fit(X, y, sample_weight=sample_weight, batch_size=64, epochs=5, verbose=0)
        return self.model
    
    def predict(self, X):
        # TODO: devuelve la clase ganadora
        return np.argmax(self.model.predict(X), axis=1)
    
    def predict_proba(self, X):
        # TODO: devuelve las probabilidades de cada clase
        return self.model.predict(X)
    
    def score(self, X, y):
        # TODO: devuelve el accuracy del clasificador
        _, acc = self.model.evaluate(X, y)
        return acc

    def __del__(self):
        del self.model
        tf.keras.backend.clear_session() # Necesario para liberar la memoria en GPU

### Entrenando el modelo

Crea una función que nos permita crear el modelo en cada iteración.

In [37]:
# TODO: Entrena el modelo
def model_func():
    return MiClasificador()

baseline_model = model_func()
baseline_model.fit(x_train, y_train)
print("Baseline Test Score: ", baseline_model.score(x_test, y_test))

/Users/circus/repos/3year2sem/maai/maai_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - accuracy: 0.7226 - loss: 2.8964
Baseline Test Score:  0.7211999893188477
